<a href="https://colab.research.google.com/github/JasmineGonzalez10/Stanford-Classes/blob/main/CS109_Challenge_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from urllib.request import urlopen
import math
import random
from scipy.stats import norm
from google.colab import files
import urllib
import io

The below is a [dataset from Kaggle](https://www.kaggle.com/datasets/hakabuk/gun-deaths-in-the-us) of gun deaths that summarizes basic information about all of the people in the US that suffered deaths at the hands of guns from 2012 to 2014.

In [92]:
df = pd.read_csv("https://raw.githubusercontent.com/JasmineGonzalez10/Computational-Modeling/master/guns.csv")
df = df.dropna()
df

,Unnamed: 0,year,month,intent,police,sex,age,race,hispanic,place,education
0,1,2012,1,Suicide,0,M,34.0,Asian/Pacific Islander,100,Home,4.0
1,2,2012,1,Suicide,0,F,21.0,White,100,Street,3.0
2,3,2012,1,Suicide,0,M,60.0,White,100,Other specified,4.0
3,4,2012,2,Suicide,0,M,64.0,White,100,Home,4.0
4,5,2012,2,Suicide,0,M,31.0,White,100,Other specified,2.0
...,...,...,...,...,...,...,...,...,...,...,...
100793,100794,2014,12,Homicide,0,M,36.0,Black,100,Home,2.0
100794,100795,2014,12,Homicide,0,M,19.0,Black,100,Street,2.0
100795,100796,2014,12,Homicide,0,M,20.0,Black,100,Street,2.0
100796,100797,2014,12,Homicide,0,M,22.0,Hispanic,260,Street,1.0


In [93]:
#generates a bernoulli vector to pass into sample mean boot
def demographic_bernoulli(column, demographic):
  result = []
  for entry in column:
    if entry == demographic:
      result.append(1)
    else:
      result.append(0)
  return result

In [94]:
#sample mean bootstrap
def sample_mean_boot(data, sample_size):
  sample_means = []
  N = sample_size
  M = len(data)
  
  for i in range(10000):
    sample = []
    #samples of size sample_size
    for i in range(N):
      sample.append(data[random.randint(0, M - 1)])
    mu = np.mean(sample)
    sample_means.append(mu)
  
  return sample_means

In [95]:
#sample median bootstrap
def sample_median_boot(data, sample_size):
  sample_medians = []
  N = sample_size
  M = len(data)
  
  for i in range(10000):
    sample = []
    #samples of size sample_size
    for i in range(N):
      sample.append(data[random.randint(0, M - 1)])
    median = np.median(sample)
    sample_medians.append(median)
  
  return sample_medians

In [96]:
#p-value for difference of means bootstrap
def pvalue_boot(demographic1, demographic2):
  N = len(demographic1)
  M = len(demographic2)
  observed_diff = abs(np.mean(demographic1) - np.mean(demographic2))
  uni_sample = demographic1 + demographic2
  count = 0
  for i in range(1000):
    resample1 = []
    resample2 = []
    for i in range(N):
      resample1.append(uni_sample[random.randint(0, len(uni_sample) - 1)])
    for i in range(M):
      resample2.append(uni_sample[random.randint(0, len(uni_sample) - 1)])
    mu1 = np.mean(resample1)
    mu2 = np.mean(resample2)
    diff = abs(mu1 - mu2)
    if diff >= observed_diff:
      count += 1
  p_value = count / 1000
  return p_value

In [97]:
sample_size = 1000

Using the functions defined above, we will compute the expected value and variance of the median age of victims of gun deaths, the expected variance of the mean percentages of male and female gun deaths as well as the p-value of the difference in mean ages of female and male US gun deaths. We will also compute the the expected variance of the mean percentages of homicide and suicide gun deaths and the p-value of the difference between the mean ages of homicide and suicide gun deaths. All of this will be done using bootstrap sampling. 

In [98]:
#expected value and variance of median age of victims of gun deaths in a group of 1000 gun deaths
age_data = []
for entry in df['age']:
  age_data.append(entry)
age_sample_medians = sample_median_boot(age_data, sample_size)
print("Expected value of the median age of victims of gun deaths in the US: ", np.nanmean(age_sample_medians))
print("Variance of the median age of victims of gun deaths in the US: ", np.nanvar(age_sample_medians))

Expected value of the median age of victims of gun deaths in the US:  42.0561
Variance of the median age of victims of gun deaths in the US:  1.2012527900000003


In [99]:
#setting up the sex-based bootstraps below
sex_data = []
for entry in df['sex']:
  sex_data.append(entry)

female_data = demographic_bernoulli(sex_data, 'F')
male_data = demographic_bernoulli(sex_data, 'M')

female_age = []
male_age = []
for i in range (len(df['sex'])):
  if df.iloc[i]['sex'] == "F":
    female_age.append(df.iloc[i]['age'])
  elif df.iloc[i]['sex'] == "M":
    male_age.append(df.iloc[i]['age'])

In [117]:
#expected variance of the mean percentage of female gun deaths in a group of 1000 gun deaths
female_sample_means = sample_mean_boot(female_data, sample_size)
print(female_sample_means)
print("Expected mean percentage of female gun deaths in the US: ", np.nanmean(female_sample_means))
print("Expected variance of the mean percentage of female gun deaths in the US: ", np.nanvar(female_sample_means))

[0.144, 0.143, 0.139, 0.149, 0.149, 0.157, 0.146, 0.154, 0.136, 0.153, 0.137, 0.151, 0.123, 0.131, 0.147, 0.15, 0.152, 0.124, 0.149, 0.144, 0.155, 0.15, 0.142, 0.138, 0.144, 0.162, 0.142, 0.159, 0.144, 0.132, 0.147, 0.153, 0.145, 0.143, 0.139, 0.133, 0.161, 0.151, 0.135, 0.136, 0.126, 0.162, 0.145, 0.132, 0.168, 0.136, 0.111, 0.148, 0.157, 0.148, 0.155, 0.139, 0.162, 0.149, 0.136, 0.168, 0.162, 0.138, 0.18, 0.143, 0.159, 0.142, 0.153, 0.132, 0.15, 0.146, 0.141, 0.137, 0.143, 0.157, 0.157, 0.152, 0.134, 0.143, 0.128, 0.161, 0.149, 0.163, 0.126, 0.133, 0.127, 0.131, 0.144, 0.15, 0.132, 0.125, 0.14, 0.138, 0.148, 0.164, 0.148, 0.137, 0.158, 0.124, 0.158, 0.173, 0.148, 0.158, 0.14, 0.159, 0.133, 0.143, 0.138, 0.142, 0.136, 0.141, 0.154, 0.171, 0.157, 0.151, 0.142, 0.126, 0.155, 0.158, 0.149, 0.149, 0.124, 0.14, 0.134, 0.153, 0.138, 0.127, 0.173, 0.136, 0.156, 0.147, 0.146, 0.142, 0.165, 0.154, 0.149, 0.14, 0.161, 0.172, 0.142, 0.142, 0.137, 0.141, 0.156, 0.136, 0.155, 0.137, 0.149, 0.138, 

In [118]:
#expected variance of the mean percentage of male gun deaths in a group of 1000 gun deaths
male_sample_means = sample_mean_boot(male_data, sample_size)
print(male_sample_means)
print("Expected mean percentage of male gun deaths in the US: ", np.nanmean(male_sample_means))
print("Expected variance of of the mean percentage of male gun deaths in the US: ", np.nanvar(male_sample_means))

[0.853, 0.868, 0.863, 0.86, 0.86, 0.848, 0.865, 0.861, 0.865, 0.859, 0.851, 0.851, 0.86, 0.86, 0.852, 0.867, 0.856, 0.876, 0.844, 0.869, 0.855, 0.876, 0.861, 0.861, 0.856, 0.862, 0.864, 0.854, 0.875, 0.872, 0.846, 0.853, 0.865, 0.838, 0.85, 0.854, 0.853, 0.851, 0.847, 0.833, 0.848, 0.857, 0.84, 0.875, 0.864, 0.862, 0.868, 0.861, 0.872, 0.858, 0.859, 0.853, 0.871, 0.844, 0.857, 0.86, 0.863, 0.845, 0.855, 0.86, 0.869, 0.865, 0.864, 0.855, 0.86, 0.858, 0.861, 0.847, 0.872, 0.873, 0.851, 0.856, 0.853, 0.852, 0.863, 0.854, 0.875, 0.857, 0.846, 0.852, 0.85, 0.854, 0.849, 0.86, 0.842, 0.865, 0.838, 0.836, 0.862, 0.84, 0.849, 0.867, 0.858, 0.864, 0.859, 0.849, 0.869, 0.863, 0.846, 0.865, 0.864, 0.866, 0.818, 0.849, 0.863, 0.849, 0.871, 0.852, 0.857, 0.858, 0.852, 0.854, 0.861, 0.86, 0.846, 0.856, 0.852, 0.84, 0.855, 0.862, 0.853, 0.866, 0.872, 0.864, 0.873, 0.86, 0.865, 0.852, 0.861, 0.849, 0.841, 0.864, 0.868, 0.867, 0.852, 0.86, 0.872, 0.863, 0.868, 0.845, 0.849, 0.847, 0.851, 0.825, 0.85, 0

In [102]:
#expected mean of female gun death age in a group of 1000 female gun deaths
female_age_means = sample_mean_boot(female_age, sample_size)
print(female_age_means)
print("Expected mean age of female gun deaths in the US: ", np.nanmean(female_age_means))

[43.877, 43.542, 43.566, 43.696, 44.188, 43.253, 43.487, 44.574, 43.41, 43.623, 43.864, 43.668, 44.223, 43.862, 44.331, 43.718, 43.275, 43.993, 43.828, 43.955, 43.869, 44.831, 42.942, 44.312, 43.668, 43.596, 43.548, 43.719, 43.747, 42.898, 43.938, 44.477, 43.63, 44.366, 43.898, 44.034, 44.543, 43.19, 43.867, 44.024, 43.067, 42.69, 44.092, 43.049, 43.921, 43.811, 43.518, 43.955, 44.122, 44.368, 44.141, 43.23, 43.386, 43.584, 43.38, 43.371, 44.368, 43.632, 44.393, 42.883, 42.924, 44.429, 44.386, 43.984, 44.026, 43.773, 44.148, 44.554, 43.904, 43.889, 43.405, 43.016, 43.878, 43.553, 44.427, 43.327, 44.036, 43.887, 44.171, 43.592, 43.212, 44.574, 43.239, 44.222, 43.812, 44.188, 44.839, 44.461, 43.875, 43.692, 43.687, 44.531, 43.202, 43.815, 43.876, 43.628, 42.733, 44.446, 43.614, 42.529, 43.804, 43.467, 43.663, 43.81, 44.221, 43.128, 43.551, 43.718, 43.5, 44.681, 45.236, 44.201, 43.732, 43.651, 43.354, 43.407, 43.685, 44.813, 43.951, 43.085, 44.046, 44.725, 44.54, 44.312, 44.058, 43.7, 44.

In [103]:
#expected mean of male gun death age in a group of 1000 male gun deaths
male_age_means = sample_mean_boot(male_age, sample_size)
print(male_age_means)
print("Expected mean age of male gun deaths in the US: ", np.nanmean(male_age_means))

[42.568, 42.689, 43.302, 44.017, 44.369, 43.627, 42.685, 44.303, 44.748, 43.555, 43.87, 44.495, 44.29, 43.84, 44.045, 44.224, 43.294, 43.869, 44.713, 44.074, 44.253, 43.894, 43.378, 44.424, 44.07, 44.162, 45.207, 43.448, 44.341, 44.12, 43.916, 44.65, 45.015, 43.154, 44.259, 44.406, 43.234, 43.48, 42.942, 45.175, 43.428, 44.194, 44.53, 44.157, 43.008, 43.813, 44.707, 43.744, 45.126, 44.68, 44.396, 43.879, 44.074, 42.88, 44.789, 43.944, 43.635, 43.145, 43.651, 43.475, 44.665, 44.687, 43.66, 43.912, 43.152, 44.653, 44.001, 43.595, 42.714, 44.083, 44.743, 43.589, 44.492, 45.939, 44.17, 43.434, 43.314, 43.404, 44.277, 44.116, 43.759, 44.827, 43.151, 44.432, 45.026, 43.565, 43.301, 44.246, 44.795, 43.716, 44.412, 43.714, 43.743, 44.597, 43.569, 43.388, 43.581, 43.932, 44.716, 42.833, 44.528, 43.83, 43.914, 44.443, 44.445, 43.445, 43.639, 44.115, 43.584, 43.255, 43.398, 44.516, 43.223, 43.233, 43.861, 44.191, 44.996, 44.798, 44.141, 44.467, 44.383, 43.672, 43.681, 44.155, 43.732, 43.693, 43.7

In [113]:
#p-value of the difference in mean ages of female and male gun deaths

sex_age_diff_pvalue = pvalue_boot(female_age, male_age)
print("Female ages: ", female_age)
print("Male ages: ", male_age)
print("Observed difference in means: ", np.nanmean(female_age) - np.nanmean(male_age))
print("p-value of the difference in mean ages of female and male gun deaths: ", sex_age_diff_pvalue)


Female ages:  [21.0, 52.0, 47.0, 25.0, 26.0, 50.0, 23.0, 62.0, 44.0, 30.0, 50.0, 44.0, 22.0, 19.0, 51.0, 48.0, 33.0, 89.0, 31.0, 31.0, 59.0, 30.0, 36.0, 53.0, 48.0, 53.0, 41.0, 20.0, 18.0, 32.0, 31.0, 37.0, 20.0, 17.0, 28.0, 24.0, 65.0, 54.0, 68.0, 31.0, 40.0, 37.0, 43.0, 46.0, 44.0, 40.0, 24.0, 69.0, 32.0, 29.0, 48.0, 30.0, 17.0, 20.0, 44.0, 55.0, 69.0, 20.0, 30.0, 39.0, 29.0, 18.0, 36.0, 28.0, 45.0, 65.0, 48.0, 25.0, 54.0, 61.0, 20.0, 51.0, 40.0, 54.0, 49.0, 41.0, 25.0, 37.0, 52.0, 46.0, 38.0, 41.0, 55.0, 73.0, 49.0, 48.0, 31.0, 46.0, 31.0, 48.0, 31.0, 65.0, 49.0, 53.0, 27.0, 33.0, 62.0, 78.0, 24.0, 52.0, 49.0, 43.0, 77.0, 35.0, 21.0, 22.0, 17.0, 40.0, 43.0, 21.0, 52.0, 42.0, 58.0, 61.0, 52.0, 52.0, 64.0, 62.0, 57.0, 61.0, 14.0, 47.0, 63.0, 69.0, 51.0, 29.0, 80.0, 58.0, 57.0, 37.0, 56.0, 43.0, 35.0, 47.0, 43.0, 36.0, 43.0, 96.0, 29.0, 70.0, 66.0, 54.0, 52.0, 20.0, 72.0, 33.0, 24.0, 34.0, 54.0, 40.0, 39.0, 16.0, 39.0, 40.0, 36.0, 40.0, 29.0, 21.0, 56.0, 14.0, 48.0, 38.0, 32.0, 35.0, 2

In [105]:
#setting up the intent-based bootstrap below
intent_data = []
for entry in df['intent']:
  intent_data.append(entry)

homicide_data = demographic_bernoulli(intent_data, 'Homicide')
suicide_data = demographic_bernoulli(intent_data, 'Suicide')

homicide_age = []
suicide_age = []
for i in range (len(df['intent'])):
  if df.iloc[i]['intent'] == "Homicide":
    homicide_age.append(df.iloc[i]['age'])
  elif df.iloc[i]['intent'] == "Suicide":
    suicide_age.append(df.iloc[i]['age'])


In [116]:
#expected variance of the mean percentage of homicide gun deaths in a group of 1000 gun deaths
homicide_sample_means = sample_mean_boot(homicide_data, sample_size)
print(homicide_sample_means)
print("Expected mean percentage of homicide gun deaths in the US: ", np.nanmean(homicide_sample_means))
print("Expected variance of of the mean percentage of homicide gun deaths in the US: ", np.nanvar(homicide_sample_means))

[0.366, 0.32, 0.306, 0.375, 0.327, 0.323, 0.334, 0.334, 0.304, 0.355, 0.317, 0.329, 0.329, 0.329, 0.353, 0.328, 0.338, 0.336, 0.334, 0.342, 0.336, 0.35, 0.346, 0.35, 0.317, 0.342, 0.34, 0.337, 0.348, 0.338, 0.332, 0.349, 0.355, 0.351, 0.336, 0.366, 0.351, 0.338, 0.324, 0.332, 0.35, 0.334, 0.351, 0.321, 0.333, 0.341, 0.341, 0.374, 0.361, 0.351, 0.345, 0.345, 0.341, 0.334, 0.334, 0.318, 0.34, 0.337, 0.332, 0.313, 0.353, 0.329, 0.311, 0.343, 0.343, 0.354, 0.326, 0.383, 0.34, 0.364, 0.363, 0.354, 0.309, 0.348, 0.341, 0.309, 0.349, 0.337, 0.342, 0.312, 0.345, 0.332, 0.341, 0.335, 0.361, 0.349, 0.318, 0.355, 0.352, 0.338, 0.329, 0.339, 0.358, 0.353, 0.332, 0.373, 0.358, 0.314, 0.337, 0.318, 0.328, 0.35, 0.307, 0.329, 0.373, 0.32, 0.346, 0.334, 0.305, 0.342, 0.318, 0.334, 0.331, 0.347, 0.349, 0.331, 0.362, 0.366, 0.36, 0.33, 0.336, 0.348, 0.348, 0.335, 0.336, 0.34, 0.355, 0.331, 0.33, 0.332, 0.346, 0.345, 0.326, 0.328, 0.345, 0.325, 0.332, 0.354, 0.366, 0.338, 0.348, 0.329, 0.356, 0.381, 0.33

In [115]:
#expected variance of the mean percentage of suicide gun deaths in a group of 1000 gun deaths
suicide_sample_means = sample_mean_boot(suicide_data, sample_size)
print(suicide_sample_means)
print("Expected mean percentage of suicide gun deaths in the US: ", np.nanmean(suicide_sample_means))
print("Expected variance of of the mean percentage of suicide gun deaths in the US: ", np.nanvar(suicide_sample_means))

[0.62, 0.645, 0.612, 0.639, 0.627, 0.63, 0.651, 0.648, 0.629, 0.647, 0.663, 0.636, 0.608, 0.628, 0.642, 0.638, 0.636, 0.629, 0.625, 0.652, 0.623, 0.606, 0.642, 0.616, 0.637, 0.637, 0.629, 0.618, 0.647, 0.645, 0.664, 0.626, 0.627, 0.636, 0.636, 0.638, 0.616, 0.632, 0.636, 0.65, 0.631, 0.628, 0.635, 0.631, 0.653, 0.654, 0.646, 0.604, 0.637, 0.622, 0.637, 0.631, 0.669, 0.643, 0.656, 0.657, 0.632, 0.649, 0.644, 0.655, 0.668, 0.622, 0.634, 0.637, 0.632, 0.646, 0.658, 0.645, 0.621, 0.634, 0.639, 0.66, 0.619, 0.659, 0.644, 0.629, 0.661, 0.647, 0.656, 0.622, 0.612, 0.664, 0.627, 0.588, 0.647, 0.635, 0.639, 0.644, 0.647, 0.629, 0.632, 0.626, 0.656, 0.621, 0.635, 0.64, 0.654, 0.623, 0.642, 0.626, 0.656, 0.645, 0.629, 0.638, 0.64, 0.664, 0.616, 0.639, 0.639, 0.627, 0.625, 0.624, 0.637, 0.651, 0.632, 0.602, 0.644, 0.645, 0.672, 0.657, 0.655, 0.624, 0.646, 0.657, 0.622, 0.635, 0.611, 0.643, 0.649, 0.646, 0.633, 0.632, 0.634, 0.624, 0.645, 0.623, 0.649, 0.637, 0.641, 0.631, 0.646, 0.641, 0.629, 0.61

In [111]:
#expected mean age of homicide gun death in a group of 1000 homicide gun deaths
homicide_age_means = sample_mean_boot(homicide_age, sample_size)
print(np.nanmean(homicide_age_means))
print("Expected mean age of homicide gun deaths in the US: ", np.nanmean(homicide_age_means))

32.5031616
Expected mean age of homicide gun deaths in the US:  32.5031616


In [109]:
#expected mean age of suicide gun death in a group of 1000 suicide gun deaths
suicide_age_means = sample_mean_boot(suicide_age, sample_size)
print(suicide_age_means)
print("Expected mean age of suicide gun deaths in the US: ", np.nanmean(suicide_age_means))

[49.025, 50.417, 49.462, 51.395, 51.312, 50.616, 49.514, 50.158, 49.935, 50.479, 49.11, 50.829, 49.782, 51.112, 48.936, 50.433, 49.98, 50.94, 49.435, 49.784, 50.185, 50.006, 50.301, 50.152, 50.462, 50.361, 51.165, 50.969, 51.514, 50.192, 50.108, 49.787, 50.84, 49.868, 49.74, 50.057, 50.167, 51.682, 50.375, 50.081, 49.803, 50.389, 50.186, 50.298, 49.885, 49.627, 49.693, 48.948, 50.965, 50.757, 50.125, 50.147, 50.647, 50.695, 50.795, 51.176, 50.911, 50.281, 51.028, 50.476, 51.16, 50.871, 50.853, 50.528, 50.999, 50.496, 50.302, 50.028, 50.314, 50.876, 49.294, 49.642, 50.66, 50.443, 50.407, 51.432, 49.722, 50.785, 51.209, 50.321, 51.643, 49.712, 50.887, 50.964, 50.967, 50.326, 51.056, 50.062, 50.252, 49.797, 50.697, 50.962, 50.589, 50.783, 50.631, 50.62, 50.569, 50.462, 50.22, 51.611, 51.19, 50.018, 50.943, 49.716, 49.619, 49.55, 50.469, 51.301, 50.838, 50.24, 50.012, 50.209, 50.044, 50.245, 51.158, 49.644, 50.936, 50.197, 50.141, 50.058, 50.066, 49.336, 50.083, 49.57, 50.891, 50.586, 49.9

In [110]:
#p-value of the difference between the mean ages of homicide and suicide gun deaths

intent_age_diff_pvalue = pvalue_boot(homicide_age, suicide_age)
print("Homicide ages: ", homicide_age)
print("Suicide ages: ", suicide_age)
print("Observed difference in means: ", np.nanmean(homicide_age) - np.nanmean(suicide_age))
print("p-value of the difference in mean ages of homicide and suicide gun deaths: ", intent_age_diff_pvalue)

Homicide ages:  [27.0, 31.0, 47.0, 51.0, 41.0, 43.0, 25.0, 26.0, 59.0, 28.0, 38.0, 40.0, 19.0, 30.0, 9.0, 27.0, 61.0, 25.0, 58.0, 55.0, 23.0, 52.0, 40.0, 45.0, 53.0, 26.0, 28.0, 25.0, 68.0, 48.0, 47.0, 62.0, 36.0, 41.0, 30.0, 39.0, 35.0, 35.0, 22.0, 23.0, 19.0, 33.0, 23.0, 49.0, 44.0, 46.0, 21.0, 42.0, 34.0, 50.0, 36.0, 24.0, 20.0, 18.0, 32.0, 31.0, 42.0, 34.0, 42.0, 47.0, 28.0, 45.0, 31.0, 46.0, 18.0, 15.0, 29.0, 17.0, 32.0, 21.0, 54.0, 24.0, 19.0, 21.0, 25.0, 30.0, 68.0, 31.0, 33.0, 48.0, 43.0, 19.0, 17.0, 44.0, 24.0, 26.0, 28.0, 36.0, 21.0, 20.0, 44.0, 17.0, 47.0, 29.0, 22.0, 28.0, 30.0, 13.0, 17.0, 20.0, 44.0, 27.0, 33.0, 32.0, 33.0, 22.0, 45.0, 53.0, 33.0, 71.0, 13.0, 32.0, 29.0, 42.0, 20.0, 18.0, 20.0, 27.0, 50.0, 31.0, 24.0, 65.0, 17.0, 58.0, 54.0, 23.0, 25.0, 34.0, 22.0, 38.0, 33.0, 33.0, 59.0, 20.0, 31.0, 40.0, 55.0, 22.0, 63.0, 20.0, 28.0, 25.0, 30.0, 57.0, 22.0, 18.0, 19.0, 25.0, 22.0, 50.0, 19.0, 65.0, 37.0, 28.0, 46.0, 12.0, 25.0, 41.0, 55.0, 42.0, 80.0, 73.0, 20.0, 59.0, 